In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from datetime import timedelta
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [2]:
df_calen = pd.read_csv('../data/calendar.csv')
df_val = pd.read_csv('../data/sales_train_validation.csv')
df_eval = pd.read_csv('../data/sales_train_evaluation.csv')
df_price = pd.read_csv('../data/sell_prices.csv')

In [3]:

def set_historic(df, idx, df_calen=df_calen, df_price=df_price):
    # # display(df)
    # idx = df['id'].str.split('_',expand=True).drop(columns=5).T.apply(('_').join).unique()[0]
    idx = ('_').join(idx.split('_')[:-1])

    # print(idx)

    df_fore_1 = df[df['id'].str.contains(idx)].melt(id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'])
    df_cal_fore = pd.merge(df_fore_1[['id','store_id','item_id','variable','value']], df_calen, left_on='variable', right_on='d')
    df_cal_fore['date'] = pd.to_datetime(df_cal_fore['date'])
    df_month = df_cal_fore.groupby([pd.Grouper(key='date', freq='ME'),'year','month']).agg({'value':'sum',}).reset_index()
    df_cal_fore_full = pd.merge(df_cal_fore, df_month.drop('date', axis=1), on=['year','month'], suffixes=('','_month'))
    df_full = pd.merge(df_cal_fore_full, df_price, on=['store_id','item_id','wm_yr_wk'])
    df_full['sales'] = df_full['value_month']*df_full['sell_price']
    df_full['date_month'] = df_full['date'].dt.strftime('%Y-%m')
    df_full['date_month'] = pd.to_datetime(df_full['date_month']) 
    # df_full['date_month'] = pd.to_datetime(df_full['date']) 

    df_full['idx'] = idx
        
    return df_full

In [4]:
df_val[(df_val['state_id'] == 'CA') & (df_val['dept_id'] == 'FOODS_1')].shape

(864, 1919)

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Linear Deterministic Process

In [6]:
idxs = df_val[(df_val['state_id'] == 'CA') & (df_val['dept_id'] == 'FOODS_1')].id.unique()#[:10]
forecast = pd.DataFrame({'idx':idxs})


In [7]:
# forecast

In [8]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression

def ldp(df_train):
    df = df_train.copy()
    df.index = pd.date_range(start=df.index[0], periods=len(df.index), freq="M")  # Adjust freq as needed
    
    # # display(df)
    
    fourier = CalendarFourier(freq="M", order=12)  # annual seasonality ending in December

    dp = DeterministicProcess(
        index=df.index,
        constant=True,               # dummy feature for bias (y-intercept)
        order=1,                     # trend (order 1 means linear)
        seasonal=True,               # weekly seasonality (indicators)
        additional_terms=[fourier],  # annual seasonality (fourier)
        drop=True,                   # drop terms to avoid collinearity
    )

    X_dp = dp.in_sample()


    y_dp = df['sales']

    model_ldp = LinearRegression(fit_intercept=True)
    _ = model_ldp.fit(X_dp, y_dp)

    # y_pred = pd.Series(model_ldp.predict(X_dp), index=y_dp.index)
    X_fore = dp.out_of_sample(steps=1)
    y_fore = pd.Series(model_ldp.predict(X_fore), index=X_fore.index)
    
    y_fore.index.name = "date_month"
    y_fore = y_fore.reset_index()
    y_fore['date_month'] = pd.to_datetime(y_fore['date_month'].dt.strftime('%Y-%m'))
    y_fore = y_fore.set_index('date_month')
    y_fore['id'] = df['id'].unique()[0]
    y_fore.rename(columns={0:'sales'}, inplace=True)
    
    return y_fore


In [9]:
df = forecast[forecast['idx'] == 'FOODS_1_001_CA_1_validation']

In [10]:
# set_historic(df_val, idx)

In [11]:
df['idx']#.values[0]

0    FOODS_1_001_CA_1_validation
Name: idx, dtype: object

In [12]:
# idx = df['idx'].values[0]
idx = 'FOODS_1_017_CA_1_validation'
df_full = set_historic(df_val, idx)


# idx = 'FOODS_1_017_CA_1_evaluation'
# df_full = set_historic(df_eval, idx)

sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
name_idx = ('_').join(idx.split('_')[:-1]) 
sales['id'] = name_idx
# df_full_2
y = sales#[['date_month','sales']].set_index('date_month').sort_index()
# y = y[:-2]
y.tail()

,sales,id
date_month,,
2015-12-01,2083.20,FOODS_1_017_CA_1
2016-01-01,2291.52,FOODS_1_017_CA_1
2016-02-01,1364.16,FOODS_1_017_CA_1
2016-03-01,1874.88,FOODS_1_017_CA_1
2016-04-01,1290.24,FOODS_1_017_CA_1


In [13]:
y.tail()

,sales,id
date_month,,
2015-12-01,2083.20,FOODS_1_017_CA_1
2016-01-01,2291.52,FOODS_1_017_CA_1
2016-02-01,1364.16,FOODS_1_017_CA_1
2016-03-01,1874.88,FOODS_1_017_CA_1
2016-04-01,1290.24,FOODS_1_017_CA_1


In [14]:
ldp(y).reset_index()
# y.reset_index().merge(ldp(y).reset_index(), on='date_month').drop_duplicates()
# pd.merge(y.reset_index(), ldp(y).reset_index(), on='date_month')#.drop_duplicates()

,date_month,sales,id
0,2016-05-01,1443.130224,FOODS_1_017_CA_1


In [15]:
value_pred = ldp(y)#.values[0]
y_pred = pd.concat([y,value_pred],axis=0)
y_o = y_pred.copy()
y_pred.tail()

,sales,id
date_month,,
2016-01-01,2291.520000,FOODS_1_017_CA_1
2016-02-01,1364.160000,FOODS_1_017_CA_1
2016-03-01,1874.880000,FOODS_1_017_CA_1
2016-04-01,1290.240000,FOODS_1_017_CA_1
2016-05-01,1443.130224,FOODS_1_017_CA_1


In [16]:
y.tail()#.index[-1]

,sales,id
date_month,,
2015-12-01,2083.20,FOODS_1_017_CA_1
2016-01-01,2291.52,FOODS_1_017_CA_1
2016-02-01,1364.16,FOODS_1_017_CA_1
2016-03-01,1874.88,FOODS_1_017_CA_1
2016-04-01,1290.24,FOODS_1_017_CA_1


In [17]:
value_pred
y_o.tail()

,sales,id
date_month,,
2016-01-01,2291.520000,FOODS_1_017_CA_1
2016-02-01,1364.160000,FOODS_1_017_CA_1
2016-03-01,1874.880000,FOODS_1_017_CA_1
2016-04-01,1290.240000,FOODS_1_017_CA_1
2016-05-01,1443.130224,FOODS_1_017_CA_1


In [18]:
idxs = df_val[(df_val['state_id'] == 'CA') & (df_val['dept_id'] == 'FOODS_1')].id.unique()#[:10]
forecast = pd.DataFrame({'idx':idxs})

In [19]:
# def forecast_calc(df):
#     idx = df['idx'].values[0]
#     df_full = set_historic(df_val, idx)

#     sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
#     name_idx = ('_').join(idx.split('_')[:-1]) 
#     sales['id'] = name_idx

#     y = sales#[['date_month','sales']].set_index('date_month').sort_index()
#     y = y[:-2]

#     try:
#         model_log_hw = ExponentialSmoothing(endog=np.log1p(y['sales']), trend='add', damped_trend=True, seasonal='add',
#                                             seasonal_periods=12, initialization_method='estimated').fit()
#         value_pred = (np.exp(model_log_hw.forecast(1))-1)
#         value_pred = pd.DataFrame(value_pred, columns=['sales'])
#         value_pred['id'] = name_idx
#         y_pred = pd.concat([y,value_pred],axis=0)

#     except:
#         value_pred = ldp(y)
#         y_pred = pd.concat([y,value_pred],axis=0)

#     y_pred.index.name = "date_month"
    
#     return y_pred.reset_index()

In [20]:
def forecast_calc(df):
    idx = df['idx'].values[0]
    df_full = set_historic(df_val, idx)

    sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
    name_idx = ('_').join(idx.split('_')[:-1]) 
    sales['id'] = name_idx

    y = sales#[['date_month','sales']].set_index('date_month').sort_index()
    y = y[:-2]

    value_pred = ldp(y)
    y_pred = pd.concat([y,value_pred],axis=0)

    y_pred.index.name = "date_month"
    
    return y_pred.reset_index()

In [21]:
forecast_calc(forecast[forecast['idx'] == 'FOODS_1_011_CA_1_validation'])


,date_month,sales,id
0,2011-01-01,20.520000,FOODS_1_011_CA_1
1,2011-02-01,1149.120000,FOODS_1_011_CA_1
2,2011-03-01,777.480000,FOODS_1_011_CA_1
3,2011-04-01,0.000000,FOODS_1_011_CA_1
4,2011-05-01,0.000000,FOODS_1_011_CA_1
...,...,...,...
58,2015-11-01,0.000000,FOODS_1_011_CA_1
59,2015-12-01,0.000000,FOODS_1_011_CA_1
60,2016-01-01,2990.880000,FOODS_1_011_CA_1
61,2016-02-01,2564.760000,FOODS_1_011_CA_1


In [22]:
forecast_1 = forecast.groupby('idx').apply(forecast_calc)

In [23]:
forecast_1.reset_index(inplace=True)

In [24]:
forecast_1.drop(columns='level_1', inplace=True)

In [25]:
forecast_1#[forecast_2['idx'] == 'FOODS_1_011_CA_1_validation']

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_validation,2011-01-01,18.00000,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_validation,2011-02-01,2240.00000,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_validation,2011-03-01,2480.00000,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_validation,2011-04-01,1380.00000,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_validation,2011-05-01,3100.00000,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_validation,2015-11-01,0.00000,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_validation,2015-12-01,0.00000,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_validation,2016-01-01,2985.92000,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_validation,2016-02-01,4027.52000,FOODS_1_219_CA_4


In [26]:
def prepare_data(df):
    idx = df['idx'].values[0]
    df_full = set_historic(df_eval, idx)

    sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
    name_idx = ('_').join(idx.split('_')[:-1]) 
    sales['id'] = name_idx

    y = sales#[['date_month','sales']].set_index('date_month').sort_index()
    y = y[:-2]

    return y.reset_index()

In [27]:
idxs = df_eval[(df_eval['state_id'] == 'CA') & (df_eval['dept_id'] == 'FOODS_1')].id.unique()#[:10]
forecast_eval = pd.DataFrame({'idx':idxs})


In [28]:
# forecast_eval

In [29]:
prepare_data(forecast_eval[forecast_eval['idx'] == 'FOODS_1_011_CA_1_evaluation'])

,date_month,sales,id
0,2011-01-01,20.52,FOODS_1_011_CA_1
1,2011-02-01,1149.12,FOODS_1_011_CA_1
2,2011-03-01,777.48,FOODS_1_011_CA_1
3,2011-04-01,0.00,FOODS_1_011_CA_1
4,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...
58,2015-11-01,0.00,FOODS_1_011_CA_1
59,2015-12-01,0.00,FOODS_1_011_CA_1
60,2016-01-01,2990.88,FOODS_1_011_CA_1
61,2016-02-01,2564.76,FOODS_1_011_CA_1


In [30]:
forecast_eval_1 = forecast_eval.groupby('idx').apply(prepare_data)

In [31]:
forecast_eval_1.reset_index(inplace=True)

In [32]:
forecast_eval_1.drop(columns='level_1', inplace=True)

In [33]:
forecast_eval_1#[forecast_2['idx'] == 'FOODS_1_011_CA_1_validation']
# forecast_1

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_evaluation,2011-01-01,18.00,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_evaluation,2011-02-01,2240.00,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_evaluation,2011-03-01,2480.00,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_evaluation,2011-04-01,1380.00,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_evaluation,2011-05-01,3100.00,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_evaluation,2015-11-01,0.00,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_evaluation,2015-12-01,0.00,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_evaluation,2016-01-01,2985.92,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_evaluation,2016-02-01,4027.52,FOODS_1_219_CA_4


In [34]:
forecast_1

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_validation,2011-01-01,18.00000,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_validation,2011-02-01,2240.00000,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_validation,2011-03-01,2480.00000,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_validation,2011-04-01,1380.00000,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_validation,2011-05-01,3100.00000,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_validation,2015-11-01,0.00000,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_validation,2015-12-01,0.00000,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_validation,2016-01-01,2985.92000,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_validation,2016-02-01,4027.52000,FOODS_1_219_CA_4


In [35]:
forecast_1[forecast_1['id'] == 'FOODS_1_011_CA_1']

,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_validation,2011-01-01,20.520000,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_validation,2011-02-01,1149.120000,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_validation,2011-03-01,777.480000,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_validation,2011-04-01,0.000000,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_validation,2011-05-01,0.000000,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_validation,2015-11-01,0.000000,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_validation,2015-12-01,0.000000,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_validation,2016-01-01,2990.880000,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_validation,2016-02-01,2564.760000,FOODS_1_011_CA_1


In [36]:
forecast_eval_1[forecast_eval_1['id'] == 'FOODS_1_011_CA_1']


,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_evaluation,2011-01-01,20.52,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_evaluation,2011-02-01,1149.12,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_evaluation,2011-03-01,777.48,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_evaluation,2011-04-01,0.00,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_evaluation,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_evaluation,2015-11-01,0.00,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_evaluation,2015-12-01,0.00,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_evaluation,2016-01-01,2990.88,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_evaluation,2016-02-01,2564.76,FOODS_1_011_CA_1


In [37]:
forecast_1['date_month'].dt.day.unique()

array([1], dtype=int32)

In [38]:
df_metrics_o = pd.merge(forecast_1[['id','date_month','sales']], forecast_eval_1[['id','date_month','sales']],
          on=['id','date_month'], suffixes=('_validation','_evaluation')).dropna()

In [39]:
df_metrics_o[df_metrics_o['date_month'] == '2016-03-01'].drop_duplicates()

,id,date_month,sales_validation,sales_evaluation
62,FOODS_1_001_CA_1,2016-03-01,1530.460961,1874.88
125,FOODS_1_001_CA_2,2016-03-01,1441.664804,1944.32
188,FOODS_1_001_CA_3,2016-03-01,2130.102704,2222.08
251,FOODS_1_001_CA_4,2016-03-01,811.966212,1111.04
314,FOODS_1_002_CA_1,2016-03-01,2064.499665,4114.32
...,...,...,...,...
45624,FOODS_1_218_CA_4,2016-03-01,6322.922034,7412.72
45687,FOODS_1_219_CA_1,2016-03-01,2457.436425,8471.68
45750,FOODS_1_219_CA_2,2016-03-01,706.347263,6319.04
45813,FOODS_1_219_CA_3,2016-03-01,1725.046168,9027.20


In [40]:
forecast_eval_1[forecast_eval_1['id'] == 'FOODS_1_011_CA_1']


,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_evaluation,2011-01-01,20.52,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_evaluation,2011-02-01,1149.12,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_evaluation,2011-03-01,777.48,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_evaluation,2011-04-01,0.00,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_evaluation,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_evaluation,2015-11-01,0.00,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_evaluation,2015-12-01,0.00,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_evaluation,2016-01-01,2990.88,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_evaluation,2016-02-01,2564.76,FOODS_1_011_CA_1


In [41]:
forecast_1[forecast_1['id'] == 'FOODS_1_011_CA_1']

,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_validation,2011-01-01,20.520000,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_validation,2011-02-01,1149.120000,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_validation,2011-03-01,777.480000,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_validation,2011-04-01,0.000000,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_validation,2011-05-01,0.000000,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_validation,2015-11-01,0.000000,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_validation,2015-12-01,0.000000,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_validation,2016-01-01,2990.880000,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_validation,2016-02-01,2564.760000,FOODS_1_011_CA_1


In [42]:
forecast_1.shape, forecast_eval_1.shape

((45877, 4), (45877, 4))

In [43]:
forecast_1['id'].nunique(), forecast_eval_1['id'].nunique()

(864, 864)

In [44]:
df = pd.DataFrame({'id':forecast_1['id'].unique()})
def metrics(df):
    idx = df['id'].values[0]
    df1 = forecast_1[forecast_1['id'] == idx]#[-1:]
    df2 = forecast_eval_1[forecast_eval_1['id'] == idx]#[-1:]
    # df1['sales'] = df1['sales'].shift(1)
    # df2['sales'] = df2['sales'].shift(1)
    df1['sales_3'] = df1['sales'].shift(1).rolling(3).mean()
    df1['sales_6'] = df1['sales'].shift(1).rolling(6).mean()
    df1['sales_12'] = df1['sales'].shift(1).rolling(12).mean()
    df3 = pd.merge(df1[['id','date_month','sales','sales_3','sales_6','sales_12']][-1:],
                    df2[['id','date_month','sales']][-1:], on=['id','date_month'], suffixes=('_validation','_evaluation'))

    # display(df1)
    return df3

In [45]:
df
metrics(df[df['id'] == 'FOODS_1_011_CA_1'])

,id,date_month,sales_validation,sales_3,sales_6,sales_12,sales_evaluation
0,FOODS_1_011_CA_1,2016-03-01,3403.590302,1851.88,1680.806667,2196.93,2160.08


In [46]:
(6.84+41.04+25.08)/3

24.319999999999997

In [47]:
df_metrics = df.groupby('id').apply(metrics)

In [48]:
df_metrics.reset_index(inplace=True,drop=True)

In [49]:
df_metrics#.drop(columns='level_1', inplace=True)

,id,date_month,sales_validation,sales_3,sales_6,sales_12,sales_evaluation
0,FOODS_1_001_CA_1,2016-03-01,1530.460961,1403.733333,1346.986667,1296.960000,1874.88
1,FOODS_1_001_CA_2,2016-03-01,1441.664804,2353.493333,2159.360000,1795.920000,1944.32
2,FOODS_1_001_CA_3,2016-03-01,2130.102704,2392.320000,2517.013333,2150.213333,2222.08
3,FOODS_1_001_CA_4,2016-03-01,811.966212,393.493333,456.213333,583.706667,1111.04
4,FOODS_1_002_CA_1,2016-03-01,2064.499665,3627.680000,3204.240000,3642.690000,4114.32
...,...,...,...,...,...,...,...
859,FOODS_1_218_CA_4,2016-03-01,6322.922034,8314.973333,9526.253333,8544.701667,7412.72
860,FOODS_1_219_CA_1,2016-03-01,2457.436425,5174.400000,4118.986667,5029.733333,8471.68
861,FOODS_1_219_CA_2,2016-03-01,706.347263,3418.986667,2905.653333,3672.293333,6319.04
862,FOODS_1_219_CA_3,2016-03-01,1725.046168,6261.826667,4831.606667,6354.716667,9027.20


In [138]:
df_metrics['eval_val'] = abs(df_metrics['sales_evaluation'] - df_metrics['sales_validation'])
df_metrics['eval_3'] = abs(df_metrics['sales_evaluation'] - df_metrics['sales_3'])
df_metrics['eval_6'] = abs(df_metrics['sales_evaluation'] - df_metrics['sales_6'])
df_metrics['eval_12'] = abs(df_metrics['sales_evaluation'] - df_metrics['sales_12'])



In [139]:
df_metrics[['eval_val','eval_3','eval_6','eval_12']]#.mean()

,eval_val,eval_3,eval_6,eval_12
0,344.419039,471.146667,527.893333,577.920000
1,502.655196,409.173333,215.040000,148.400000
2,91.977296,170.240000,294.933333,71.866667
3,299.073788,717.546667,654.826667,527.333333
4,2049.820335,486.640000,910.080000,471.630000
...,...,...,...,...
859,1089.797966,902.253333,2113.533333,1131.981667
860,6014.243575,3297.280000,4352.693333,3441.946667
861,5612.692737,2900.053333,3413.386667,2646.746667
862,7302.153832,2765.373333,4195.593333,2672.483333


In [145]:
df_metrics[['eval_val','eval_3','eval_6','eval_12']].mean().to_frame(name='LDP')


,LDP
eval_val,2004.289718
eval_3,1549.630432
eval_6,1542.074668
eval_12,1636.675887


# Holtz-Winters

In [53]:
idxs = df_val[(df_val['state_id'] == 'CA') & (df_val['dept_id'] == 'FOODS_1')].id.unique()#[:10]
forecast = pd.DataFrame({'idx':idxs})

In [54]:
def forecast_calc(df):
    idx = df['idx'].values[0]
    df_full = set_historic(df_val, idx)

    sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
    name_idx = ('_').join(idx.split('_')[:-1]) 
    sales['id'] = name_idx

    y = sales#[['date_month','sales']].set_index('date_month').sort_index()
    y = y[:-2]

    try:
        model_log_hw = ExponentialSmoothing(endog=np.log1p(y['sales']), trend='add', damped_trend=True, seasonal='add',
                                            seasonal_periods=12, initialization_method='estimated').fit()
        value_pred = (np.exp(model_log_hw.forecast(1))-1)
        value_pred = pd.DataFrame(value_pred, columns=['sales'])
        value_pred['id'] = name_idx
        y_pred = pd.concat([y,value_pred],axis=0)

    except:
        value_pred = ldp(y)
        y_pred = pd.concat([y,value_pred],axis=0)

    y_pred.index.name = "date_month"
    
    return y_pred.reset_index()

In [55]:
# def forecast_calc(df):
#     idx = df['idx'].values[0]
#     df_full = set_historic(df_val, idx)

#     sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
#     name_idx = ('_').join(idx.split('_')[:-1]) 
#     sales['id'] = name_idx

#     y = sales#[['date_month','sales']].set_index('date_month').sort_index()
#     y = y[:-2]

#     model_log_hw = ExponentialSmoothing(endog=np.log1p(y['sales']), trend='add', damped_trend=True, seasonal='add',
#                                             seasonal_periods=12, initialization_method='estimated').fit()
#     value_pred = (np.exp(model_log_hw.forecast(1))-1)
#     value_pred = pd.DataFrame(value_pred, columns=['sales'])
#     value_pred['id'] = name_idx
#     y_pred = pd.concat([y,value_pred],axis=0)

#     y_pred.index.name = "date_month"
    
#     return y_pred.reset_index()

In [56]:
forecast_calc(forecast[forecast['idx'] == 'FOODS_1_011_CA_1_validation'])


,date_month,sales,id
0,2011-01-01,20.520000,FOODS_1_011_CA_1
1,2011-02-01,1149.120000,FOODS_1_011_CA_1
2,2011-03-01,777.480000,FOODS_1_011_CA_1
3,2011-04-01,0.000000,FOODS_1_011_CA_1
4,2011-05-01,0.000000,FOODS_1_011_CA_1
...,...,...,...
58,2015-11-01,0.000000,FOODS_1_011_CA_1
59,2015-12-01,0.000000,FOODS_1_011_CA_1
60,2016-01-01,2990.880000,FOODS_1_011_CA_1
61,2016-02-01,2564.760000,FOODS_1_011_CA_1


In [57]:
forecast_3 = forecast.groupby('idx').apply(forecast_calc)

In [58]:
forecast_3.reset_index(inplace=True)

In [59]:
forecast_3.drop(columns='level_1', inplace=True)

In [60]:
forecast_3#[forecast_2['idx'] == 'FOODS_1_011_CA_1_validation']

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_validation,2011-01-01,18.000000,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_validation,2011-02-01,2240.000000,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_validation,2011-03-01,2480.000000,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_validation,2011-04-01,1380.000000,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_validation,2011-05-01,3100.000000,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_validation,2015-11-01,0.000000,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_validation,2015-12-01,0.000000,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_validation,2016-01-01,2985.920000,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_validation,2016-02-01,4027.520000,FOODS_1_219_CA_4


In [61]:
def prepare_data(df):
    idx = df['idx'].values[0]
    df_full = set_historic(df_eval, idx)

    sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
    name_idx = ('_').join(idx.split('_')[:-1]) 
    sales['id'] = name_idx

    y = sales#[['date_month','sales']].set_index('date_month').sort_index()
    y = y[:-2]

    return y.reset_index()

In [62]:
idxs = df_eval[(df_eval['state_id'] == 'CA') & (df_eval['dept_id'] == 'FOODS_1')].id.unique()#[:10]
forecast_eval = pd.DataFrame({'idx':idxs})


In [63]:
# forecast_eval

In [64]:
prepare_data(forecast_eval[forecast_eval['idx'] == 'FOODS_1_011_CA_1_evaluation'])

,date_month,sales,id
0,2011-01-01,20.52,FOODS_1_011_CA_1
1,2011-02-01,1149.12,FOODS_1_011_CA_1
2,2011-03-01,777.48,FOODS_1_011_CA_1
3,2011-04-01,0.00,FOODS_1_011_CA_1
4,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...
58,2015-11-01,0.00,FOODS_1_011_CA_1
59,2015-12-01,0.00,FOODS_1_011_CA_1
60,2016-01-01,2990.88,FOODS_1_011_CA_1
61,2016-02-01,2564.76,FOODS_1_011_CA_1


In [65]:
forecast_eval_3 = forecast_eval.groupby('idx').apply(prepare_data)

In [66]:
forecast_eval_3.reset_index(inplace=True)

In [67]:
forecast_eval_3.drop(columns='level_1', inplace=True)

In [68]:
forecast_eval_3#[forecast_2['idx'] == 'FOODS_1_011_CA_1_validation']
# forecast_1

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_evaluation,2011-01-01,18.00,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_evaluation,2011-02-01,2240.00,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_evaluation,2011-03-01,2480.00,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_evaluation,2011-04-01,1380.00,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_evaluation,2011-05-01,3100.00,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_evaluation,2015-11-01,0.00,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_evaluation,2015-12-01,0.00,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_evaluation,2016-01-01,2985.92,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_evaluation,2016-02-01,4027.52,FOODS_1_219_CA_4


In [69]:
forecast_3

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_validation,2011-01-01,18.000000,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_validation,2011-02-01,2240.000000,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_validation,2011-03-01,2480.000000,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_validation,2011-04-01,1380.000000,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_validation,2011-05-01,3100.000000,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_validation,2015-11-01,0.000000,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_validation,2015-12-01,0.000000,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_validation,2016-01-01,2985.920000,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_validation,2016-02-01,4027.520000,FOODS_1_219_CA_4


In [70]:
forecast_3[forecast_3['id'] == 'FOODS_1_011_CA_1']

,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_validation,2011-01-01,20.520000,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_validation,2011-02-01,1149.120000,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_validation,2011-03-01,777.480000,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_validation,2011-04-01,0.000000,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_validation,2011-05-01,0.000000,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_validation,2015-11-01,0.000000,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_validation,2015-12-01,0.000000,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_validation,2016-01-01,2990.880000,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_validation,2016-02-01,2564.760000,FOODS_1_011_CA_1


In [71]:
forecast_eval_3[forecast_eval_3['id'] == 'FOODS_1_011_CA_1']


,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_evaluation,2011-01-01,20.52,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_evaluation,2011-02-01,1149.12,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_evaluation,2011-03-01,777.48,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_evaluation,2011-04-01,0.00,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_evaluation,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_evaluation,2015-11-01,0.00,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_evaluation,2015-12-01,0.00,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_evaluation,2016-01-01,2990.88,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_evaluation,2016-02-01,2564.76,FOODS_1_011_CA_1


In [72]:
forecast_3['date_month'].dt.day.unique()

array([1], dtype=int32)

In [73]:
df_metrics_o3 = pd.merge(forecast_3[['id','date_month','sales']], forecast_eval_3[['id','date_month','sales']],
          on=['id','date_month'], suffixes=('_validation','_evaluation')).dropna()

In [74]:
df_metrics_o3[df_metrics_o3['date_month'] == '2016-03-01'].drop_duplicates()

,id,date_month,sales_validation,sales_evaluation
62,FOODS_1_001_CA_1,2016-03-01,1488.119722,1874.88
125,FOODS_1_001_CA_2,2016-03-01,1785.081894,1944.32
188,FOODS_1_001_CA_3,2016-03-01,3170.909962,2222.08
251,FOODS_1_001_CA_4,2016-03-01,0.448867,1111.04
314,FOODS_1_002_CA_1,2016-03-01,15944.811276,4114.32
...,...,...,...,...
45624,FOODS_1_218_CA_4,2016-03-01,10844.039981,7412.72
45687,FOODS_1_219_CA_1,2016-03-01,8.293570,8471.68
45750,FOODS_1_219_CA_2,2016-03-01,5.311702,6319.04
45813,FOODS_1_219_CA_3,2016-03-01,333.924168,9027.20


In [75]:
forecast_eval_3[forecast_eval_3['id'] == 'FOODS_1_011_CA_1']


,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_evaluation,2011-01-01,20.52,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_evaluation,2011-02-01,1149.12,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_evaluation,2011-03-01,777.48,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_evaluation,2011-04-01,0.00,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_evaluation,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_evaluation,2015-11-01,0.00,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_evaluation,2015-12-01,0.00,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_evaluation,2016-01-01,2990.88,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_evaluation,2016-02-01,2564.76,FOODS_1_011_CA_1


In [76]:
forecast_3[forecast_3['id'] == 'FOODS_1_011_CA_1']

,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_validation,2011-01-01,20.520000,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_validation,2011-02-01,1149.120000,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_validation,2011-03-01,777.480000,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_validation,2011-04-01,0.000000,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_validation,2011-05-01,0.000000,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_validation,2015-11-01,0.000000,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_validation,2015-12-01,0.000000,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_validation,2016-01-01,2990.880000,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_validation,2016-02-01,2564.760000,FOODS_1_011_CA_1


In [77]:
forecast_3.shape, forecast_eval_3.shape

((45877, 4), (45877, 4))

In [78]:
forecast_3['id'].nunique(), forecast_eval_3['id'].nunique()

(864, 864)

In [79]:
df = pd.DataFrame({'id':forecast_3['id'].unique()})
def metrics(df):
    idx = df['id'].values[0]
    df1 = forecast_3[forecast_3['id'] == idx]#[-1:]
    df2 = forecast_eval_3[forecast_eval_3['id'] == idx]#[-1:]
    # df1['sales'] = df1['sales'].shift(1)
    # df2['sales'] = df2['sales'].shift(1)
    df1['sales_3'] = df1['sales'].shift(1).rolling(3).mean()
    df1['sales_6'] = df1['sales'].shift(1).rolling(6).mean()
    df1['sales_12'] = df1['sales'].shift(1).rolling(12).mean()
    df3 = pd.merge(df1[['id','date_month','sales','sales_3','sales_6','sales_12']][-1:],
                    df2[['id','date_month','sales']][-1:], on=['id','date_month'], suffixes=('_validation','_evaluation'))

    # display(df1)
    return df3

In [80]:
df
metrics(df[df['id'] == 'FOODS_1_011_CA_1'])

,id,date_month,sales_validation,sales_3,sales_6,sales_12,sales_evaluation
0,FOODS_1_011_CA_1,2016-03-01,3358.254672,1851.88,1680.806667,2196.93,2160.08


In [81]:
(6.84+41.04+25.08)/3

24.319999999999997

In [82]:
df_metrics_3 = df.groupby('id').apply(metrics)

In [83]:
df_metrics_3.reset_index(inplace=True,drop=True)

In [84]:
df_metrics_3#.drop(columns='level_1', inplace=True)

,id,date_month,sales_validation,sales_3,sales_6,sales_12,sales_evaluation
0,FOODS_1_001_CA_1,2016-03-01,1488.119722,1403.733333,1346.986667,1296.960000,1874.88
1,FOODS_1_001_CA_2,2016-03-01,1785.081894,2353.493333,2159.360000,1795.920000,1944.32
2,FOODS_1_001_CA_3,2016-03-01,3170.909962,2392.320000,2517.013333,2150.213333,2222.08
3,FOODS_1_001_CA_4,2016-03-01,0.448867,393.493333,456.213333,583.706667,1111.04
4,FOODS_1_002_CA_1,2016-03-01,15944.811276,3627.680000,3204.240000,3642.690000,4114.32
...,...,...,...,...,...,...,...
859,FOODS_1_218_CA_4,2016-03-01,10844.039981,8314.973333,9526.253333,8544.701667,7412.72
860,FOODS_1_219_CA_1,2016-03-01,8.293570,5174.400000,4118.986667,5029.733333,8471.68
861,FOODS_1_219_CA_2,2016-03-01,5.311702,3418.986667,2905.653333,3672.293333,6319.04
862,FOODS_1_219_CA_3,2016-03-01,333.924168,6261.826667,4831.606667,6354.716667,9027.20


In [135]:
df_metrics_3['eval_val'] = abs(df_metrics_3['sales_evaluation'] - df_metrics_3['sales_validation'])
df_metrics_3['eval_3'] = abs(df_metrics_3['sales_evaluation'] - df_metrics_3['sales_3'])
df_metrics_3['eval_6'] = abs(df_metrics_3['sales_evaluation'] - df_metrics_3['sales_6'])
df_metrics_3['eval_12'] = abs(df_metrics_3['sales_evaluation'] - df_metrics_3['sales_12'])



In [136]:
df_metrics_3[['eval_val','eval_3','eval_6','eval_12']]#.mean()

,eval_val,eval_3,eval_6,eval_12
0,386.760278,471.146667,527.893333,577.920000
1,159.238106,409.173333,215.040000,148.400000
2,948.829962,170.240000,294.933333,71.866667
3,1110.591133,717.546667,654.826667,527.333333
4,11830.491276,486.640000,910.080000,471.630000
...,...,...,...,...
859,3431.319981,902.253333,2113.533333,1131.981667
860,8463.386430,3297.280000,4352.693333,3441.946667
861,6313.728298,2900.053333,3413.386667,2646.746667
862,8693.275832,2765.373333,4195.593333,2672.483333


In [147]:
df_metrics_3[['eval_val','eval_3','eval_6','eval_12']].mean().to_frame(name='Exp Smooth')


,Exp Smooth
eval_val,3028.446882
eval_3,1549.630432
eval_6,1542.074668
eval_12,1636.675887


# Arima

In [91]:
from statsmodels.tsa.arima.model import ARIMA

In [92]:
idxs = df_val[(df_val['state_id'] == 'CA') & (df_val['dept_id'] == 'FOODS_1')].id.unique()#[:10]
forecast = pd.DataFrame({'idx':idxs})

In [93]:
# def forecast_calc(df):
#     idx = df['idx'].values[0]
#     df_full = set_historic(df_val, idx)

#     sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
#     name_idx = ('_').join(idx.split('_')[:-1]) 
#     sales['id'] = name_idx

#     y = sales#[['date_month','sales']].set_index('date_month').sort_index()
#     y = y[:-2]

#     try:
#         model_log_hw = ExponentialSmoothing(endog=np.log1p(y['sales']), trend='add', damped_trend=True, seasonal='add',
#                                             seasonal_periods=12, initialization_method='estimated').fit()
#         value_pred = (np.exp(model_log_hw.forecast(1))-1)
#         value_pred = pd.DataFrame(value_pred, columns=['sales'])
#         value_pred['id'] = name_idx
#         y_pred = pd.concat([y,value_pred],axis=0)

#     except:
#         value_pred = ldp(y)
#         y_pred = pd.concat([y,value_pred],axis=0)

#     y_pred.index.name = "date_month"
    
#     return y_pred.reset_index()

In [94]:
def forecast_calc(df):
    idx = df['idx'].values[0]
    df_full = set_historic(df_val, idx)

    sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
    name_idx = ('_').join(idx.split('_')[:-1]) 
    sales['id'] = name_idx

    y = sales#[['date_month','sales']].set_index('date_month').sort_index()
    y = y[:-2]

    arima_mod = ARIMA(endog=y['sales'], order=(1,1,0)).fit() #(6,5,0), (1,1,0)
    value_pred = arima_mod.forecast(1)
    value_pred = pd.DataFrame(value_pred, columns=['sales'])
    value_pred['id'] = name_idx
    y_pred = pd.concat([y,value_pred],axis=0)

    y_pred.index.name = "date_month"
    
    return y_pred.reset_index()

In [95]:
forecast_calc(forecast[forecast['idx'] == 'FOODS_1_011_CA_1_validation'])


,date_month,sales,id
0,2011-01-01,20.5200,FOODS_1_011_CA_1
1,2011-02-01,1149.1200,FOODS_1_011_CA_1
2,2011-03-01,777.4800,FOODS_1_011_CA_1
3,2011-04-01,0.0000,FOODS_1_011_CA_1
4,2011-05-01,0.0000,FOODS_1_011_CA_1
...,...,...,...
58,2015-11-01,0.0000,FOODS_1_011_CA_1
59,2015-12-01,0.0000,FOODS_1_011_CA_1
60,2016-01-01,2990.8800,FOODS_1_011_CA_1
61,2016-02-01,2564.7600,FOODS_1_011_CA_1


In [97]:
forecast_5 = forecast.groupby('idx').apply(forecast_calc)

In [98]:
forecast_5.reset_index(inplace=True)

In [99]:
forecast_5.drop(columns='level_1', inplace=True)

In [100]:
forecast_5#[forecast_2['idx'] == 'FOODS_1_011_CA_1_validation']

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_validation,2011-01-01,18.000000,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_validation,2011-02-01,2240.000000,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_validation,2011-03-01,2480.000000,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_validation,2011-04-01,1380.000000,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_validation,2011-05-01,3100.000000,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_validation,2015-11-01,0.000000,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_validation,2015-12-01,0.000000,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_validation,2016-01-01,2985.920000,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_validation,2016-02-01,4027.520000,FOODS_1_219_CA_4


In [101]:
def prepare_data(df):
    idx = df['idx'].values[0]
    df_full = set_historic(df_eval, idx)

    sales = df_full.groupby('date_month').agg({'sales':'sum'})#.plot()
    name_idx = ('_').join(idx.split('_')[:-1]) 
    sales['id'] = name_idx

    y = sales#[['date_month','sales']].set_index('date_month').sort_index()
    y = y[:-2]

    return y.reset_index()

In [102]:
idxs = df_eval[(df_eval['state_id'] == 'CA') & (df_eval['dept_id'] == 'FOODS_1')].id.unique()#[:10]
forecast_eval = pd.DataFrame({'idx':idxs})


In [103]:
# forecast_eval

In [104]:
prepare_data(forecast_eval[forecast_eval['idx'] == 'FOODS_1_011_CA_1_evaluation'])

,date_month,sales,id
0,2011-01-01,20.52,FOODS_1_011_CA_1
1,2011-02-01,1149.12,FOODS_1_011_CA_1
2,2011-03-01,777.48,FOODS_1_011_CA_1
3,2011-04-01,0.00,FOODS_1_011_CA_1
4,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...
58,2015-11-01,0.00,FOODS_1_011_CA_1
59,2015-12-01,0.00,FOODS_1_011_CA_1
60,2016-01-01,2990.88,FOODS_1_011_CA_1
61,2016-02-01,2564.76,FOODS_1_011_CA_1


In [105]:
forecast_eval_5 = forecast_eval.groupby('idx').apply(prepare_data)

In [106]:
forecast_eval_5.reset_index(inplace=True)

In [107]:
forecast_eval_5.drop(columns='level_1', inplace=True)

In [108]:
forecast_eval_5#[forecast_2['idx'] == 'FOODS_1_011_CA_1_validation']
# forecast_1

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_evaluation,2011-01-01,18.00,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_evaluation,2011-02-01,2240.00,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_evaluation,2011-03-01,2480.00,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_evaluation,2011-04-01,1380.00,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_evaluation,2011-05-01,3100.00,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_evaluation,2015-11-01,0.00,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_evaluation,2015-12-01,0.00,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_evaluation,2016-01-01,2985.92,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_evaluation,2016-02-01,4027.52,FOODS_1_219_CA_4


In [109]:
forecast_5

,idx,date_month,sales,id
0,FOODS_1_001_CA_1_validation,2011-01-01,18.000000,FOODS_1_001_CA_1
1,FOODS_1_001_CA_1_validation,2011-02-01,2240.000000,FOODS_1_001_CA_1
2,FOODS_1_001_CA_1_validation,2011-03-01,2480.000000,FOODS_1_001_CA_1
3,FOODS_1_001_CA_1_validation,2011-04-01,1380.000000,FOODS_1_001_CA_1
4,FOODS_1_001_CA_1_validation,2011-05-01,3100.000000,FOODS_1_001_CA_1
...,...,...,...,...
45872,FOODS_1_219_CA_4_validation,2015-11-01,0.000000,FOODS_1_219_CA_4
45873,FOODS_1_219_CA_4_validation,2015-12-01,0.000000,FOODS_1_219_CA_4
45874,FOODS_1_219_CA_4_validation,2016-01-01,2985.920000,FOODS_1_219_CA_4
45875,FOODS_1_219_CA_4_validation,2016-02-01,4027.520000,FOODS_1_219_CA_4


In [110]:
forecast_5[forecast_5['id'] == 'FOODS_1_011_CA_1']

,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_validation,2011-01-01,20.5200,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_validation,2011-02-01,1149.1200,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_validation,2011-03-01,777.4800,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_validation,2011-04-01,0.0000,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_validation,2011-05-01,0.0000,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_validation,2015-11-01,0.0000,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_validation,2015-12-01,0.0000,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_validation,2016-01-01,2990.8800,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_validation,2016-02-01,2564.7600,FOODS_1_011_CA_1


In [111]:
forecast_eval_5[forecast_eval_5['id'] == 'FOODS_1_011_CA_1']


,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_evaluation,2011-01-01,20.52,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_evaluation,2011-02-01,1149.12,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_evaluation,2011-03-01,777.48,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_evaluation,2011-04-01,0.00,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_evaluation,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_evaluation,2015-11-01,0.00,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_evaluation,2015-12-01,0.00,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_evaluation,2016-01-01,2990.88,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_evaluation,2016-02-01,2564.76,FOODS_1_011_CA_1


In [112]:
forecast_5['date_month'].dt.day.unique()

array([1], dtype=int32)

In [113]:
df_metrics_o5 = pd.merge(forecast_5[['id','date_month','sales']], forecast_eval_5[['id','date_month','sales']],
          on=['id','date_month'], suffixes=('_validation','_evaluation')).dropna()

In [114]:
df_metrics_o5[df_metrics_o5['date_month'] == '2016-03-01'].drop_duplicates()

,id,date_month,sales_validation,sales_evaluation
62,FOODS_1_001_CA_1,2016-03-01,1285.982452,1874.88
125,FOODS_1_001_CA_2,2016-03-01,2270.221719,1944.32
188,FOODS_1_001_CA_3,2016-03-01,2471.558296,2222.08
251,FOODS_1_001_CA_4,2016-03-01,37.958581,1111.04
314,FOODS_1_002_CA_1,2016-03-01,3568.768063,4114.32
...,...,...,...,...
45624,FOODS_1_218_CA_4,2016-03-01,9215.932797,7412.72
45687,FOODS_1_219_CA_1,2016-03-01,8482.579610,8471.68
45750,FOODS_1_219_CA_2,2016-03-01,5341.540919,6319.04
45813,FOODS_1_219_CA_3,2016-03-01,10065.264303,9027.20


In [115]:
forecast_eval_5[forecast_eval_5['id'] == 'FOODS_1_011_CA_1']


,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_evaluation,2011-01-01,20.52,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_evaluation,2011-02-01,1149.12,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_evaluation,2011-03-01,777.48,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_evaluation,2011-04-01,0.00,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_evaluation,2011-05-01,0.00,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_evaluation,2015-11-01,0.00,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_evaluation,2015-12-01,0.00,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_evaluation,2016-01-01,2990.88,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_evaluation,2016-02-01,2564.76,FOODS_1_011_CA_1


In [116]:
forecast_5[forecast_5['id'] == 'FOODS_1_011_CA_1']

,idx,date_month,sales,id
2073,FOODS_1_011_CA_1_validation,2011-01-01,20.5200,FOODS_1_011_CA_1
2074,FOODS_1_011_CA_1_validation,2011-02-01,1149.1200,FOODS_1_011_CA_1
2075,FOODS_1_011_CA_1_validation,2011-03-01,777.4800,FOODS_1_011_CA_1
2076,FOODS_1_011_CA_1_validation,2011-04-01,0.0000,FOODS_1_011_CA_1
2077,FOODS_1_011_CA_1_validation,2011-05-01,0.0000,FOODS_1_011_CA_1
...,...,...,...,...
2131,FOODS_1_011_CA_1_validation,2015-11-01,0.0000,FOODS_1_011_CA_1
2132,FOODS_1_011_CA_1_validation,2015-12-01,0.0000,FOODS_1_011_CA_1
2133,FOODS_1_011_CA_1_validation,2016-01-01,2990.8800,FOODS_1_011_CA_1
2134,FOODS_1_011_CA_1_validation,2016-02-01,2564.7600,FOODS_1_011_CA_1


In [117]:
forecast_5.shape, forecast_eval_5.shape

((45877, 4), (45877, 4))

In [118]:
forecast_5['id'].nunique(), forecast_eval_5['id'].nunique()

(864, 864)

In [119]:
df = pd.DataFrame({'id':forecast_5['id'].unique()})
def metrics(df):
    idx = df['id'].values[0]
    df1 = forecast_5[forecast_5['id'] == idx]#[-1:]
    df2 = forecast_eval_5[forecast_eval_5['id'] == idx]#[-1:]
    # df1['sales'] = df1['sales'].shift(1)
    # df2['sales'] = df2['sales'].shift(1)
    df1['sales_3'] = df1['sales'].shift(1).rolling(3).mean()
    df1['sales_6'] = df1['sales'].shift(1).rolling(6).mean()
    df1['sales_12'] = df1['sales'].shift(1).rolling(12).mean()
    df3 = pd.merge(df1[['id','date_month','sales','sales_3','sales_6','sales_12']][-1:],
                    df2[['id','date_month','sales']][-1:], on=['id','date_month'], suffixes=('_validation','_evaluation'))

    # display(df1)
    return df3

In [120]:
df
metrics(df[df['id'] == 'FOODS_1_011_CA_1'])

,id,date_month,sales_validation,sales_3,sales_6,sales_12,sales_evaluation
0,FOODS_1_011_CA_1,2016-03-01,2528.0904,1851.88,1680.806667,2196.93,2160.08


In [121]:
(6.84+41.04+25.08)/3

24.319999999999997

In [122]:
df_metrics_5 = df.groupby('id').apply(metrics)

In [123]:
df_metrics_5.reset_index(inplace=True,drop=True)

In [124]:
df_metrics_5#.drop(columns='level_1', inplace=True)

,id,date_month,sales_validation,sales_3,sales_6,sales_12,sales_evaluation
0,FOODS_1_001_CA_1,2016-03-01,1285.982452,1403.733333,1346.986667,1296.960000,1874.88
1,FOODS_1_001_CA_2,2016-03-01,2270.221719,2353.493333,2159.360000,1795.920000,1944.32
2,FOODS_1_001_CA_3,2016-03-01,2471.558296,2392.320000,2517.013333,2150.213333,2222.08
3,FOODS_1_001_CA_4,2016-03-01,37.958581,393.493333,456.213333,583.706667,1111.04
4,FOODS_1_002_CA_1,2016-03-01,3568.768063,3627.680000,3204.240000,3642.690000,4114.32
...,...,...,...,...,...,...,...
859,FOODS_1_218_CA_4,2016-03-01,9215.932797,8314.973333,9526.253333,8544.701667,7412.72
860,FOODS_1_219_CA_1,2016-03-01,8482.579610,5174.400000,4118.986667,5029.733333,8471.68
861,FOODS_1_219_CA_2,2016-03-01,5341.540919,3418.986667,2905.653333,3672.293333,6319.04
862,FOODS_1_219_CA_3,2016-03-01,10065.264303,6261.826667,4831.606667,6354.716667,9027.20


In [128]:
df_metrics_5['eval_val'] = abs(df_metrics_5['sales_evaluation'] - df_metrics_5['sales_validation'])
df_metrics_5['eval_3'] = abs(df_metrics_5['sales_evaluation'] - df_metrics_5['sales_3'])
df_metrics_5['eval_6'] = abs(df_metrics_5['sales_evaluation'] - df_metrics_5['sales_6'])
df_metrics_5['eval_12'] = abs(df_metrics_5['sales_evaluation'] - df_metrics_5['sales_12'])



In [131]:
df_metrics_5[['eval_val','eval_3','eval_6','eval_12']]#.mean()

,eval_val,eval_3,eval_6,eval_12
0,588.897548,471.146667,527.893333,577.920000
1,325.901719,409.173333,215.040000,148.400000
2,249.478296,170.240000,294.933333,71.866667
3,1073.081419,717.546667,654.826667,527.333333
4,545.551937,486.640000,910.080000,471.630000
...,...,...,...,...
859,1803.212797,902.253333,2113.533333,1131.981667
860,10.899610,3297.280000,4352.693333,3441.946667
861,977.499081,2900.053333,3413.386667,2646.746667
862,1038.064303,2765.373333,4195.593333,2672.483333


In [148]:
df_metrics_5[['eval_val','eval_3','eval_6','eval_12']].mean().to_frame(name='ARIMA')


,ARIMA
eval_val,1281.008739
eval_3,1549.630432
eval_6,1542.074668
eval_12,1636.675887


In [188]:
df_metrics_5[df_metrics_5['eval_val'].lt(df_metrics_5[['eval_3','eval_6','eval_12']].mean(axis=1))  & df_metrics_5['sales_evaluation'].gt(0)][['id','sales_validation','sales_evaluation','eval_val','eval_3','eval_6','eval_12']]
df_metrics_5.loc[(df_metrics_5['eval_val'] < df_metrics_5[['eval_3']].mean(axis=1)) & df_metrics_5['sales_evaluation'].gt(0)][['id','sales_validation','sales_evaluation','eval_val','eval_3','eval_6','eval_12']]


,id,sales_validation,sales_evaluation,eval_val,eval_3,eval_6,eval_12
1,FOODS_1_001_CA_2,2270.221719,1944.32,325.901719,409.173333,215.040000,148.400000
6,FOODS_1_002_CA_3,471.500610,4995.96,4524.459390,4714.720000,4807.940000,2844.000000
8,FOODS_1_003_CA_1,2404.713795,2703.51,298.796205,476.963333,542.101667,456.237500
14,FOODS_1_004_CA_3,2626.977103,93.00,2533.977103,10223.693333,6302.106667,17962.170000
16,FOODS_1_005_CA_1,4261.110670,4389.60,128.489330,1256.700000,625.400000,466.100000
...,...,...,...,...,...,...,...
858,FOODS_1_218_CA_3,13184.742558,13610.24,425.497442,1033.573333,232.260000,1050.886667
860,FOODS_1_219_CA_1,8482.579610,8471.68,10.899610,3297.280000,4352.693333,3441.946667
861,FOODS_1_219_CA_2,5341.540919,6319.04,977.499081,2900.053333,3413.386667,2646.746667
862,FOODS_1_219_CA_3,10065.264303,9027.20,1038.064303,2765.373333,4195.593333,2672.483333


In [193]:
df_metrics_5.loc[df_metrics_5['eval_val'].lt(df_metrics_5[['eval_3']].mean(axis=1))
                  & df_metrics_5['eval_val'].lt(df_metrics_5[['eval_6']].mean(axis=1))
                  & df_metrics_5['eval_val'].lt(df_metrics_5[['eval_12']].mean(axis=1))
                  & df_metrics_5['sales_evaluation'].gt(0)][['id','sales_validation','sales_evaluation','eval_val','eval_3','eval_6','eval_12']]


,id,sales_validation,sales_evaluation,eval_val,eval_3,eval_6,eval_12
8,FOODS_1_003_CA_1,2404.713795,2703.51,298.796205,476.963333,542.101667,456.237500
14,FOODS_1_004_CA_3,2626.977103,93.00,2533.977103,10223.693333,6302.106667,17962.170000
16,FOODS_1_005_CA_1,4261.110670,4389.60,128.489330,1256.700000,625.400000,466.100000
23,FOODS_1_006_CA_4,2972.259459,3382.72,410.460541,611.600000,1131.026667,1456.446667
26,FOODS_1_008_CA_3,658.364054,438.96,219.404054,534.540000,618.320000,678.500000
...,...,...,...,...,...,...,...
857,FOODS_1_218_CA_2,12197.117288,8901.34,3295.777288,5952.193333,4555.203333,4161.488333
860,FOODS_1_219_CA_1,8482.579610,8471.68,10.899610,3297.280000,4352.693333,3441.946667
861,FOODS_1_219_CA_2,5341.540919,6319.04,977.499081,2900.053333,3413.386667,2646.746667
862,FOODS_1_219_CA_3,10065.264303,9027.20,1038.064303,2765.373333,4195.593333,2672.483333


In [189]:
df_metrics_5.loc[(df_metrics_5['eval_val'].lt(df_metrics_5[['eval_3']].mean(axis=1))
                  | df_metrics_5['eval_val'].lt(df_metrics_5[['eval_6']].mean(axis=1))
                  | df_metrics_5['eval_val'].lt(df_metrics_5[['eval_12']].mean(axis=1)))
                  & df_metrics_5['sales_evaluation'].gt(0)][['id','sales_validation','sales_evaluation','eval_val','eval_3','eval_6','eval_12']]


,id,sales_validation,sales_evaluation,eval_val,eval_3,eval_6,eval_12
1,FOODS_1_001_CA_2,2270.221719,1944.32,325.901719,409.173333,215.040000,148.400000
2,FOODS_1_001_CA_3,2471.558296,2222.08,249.478296,170.240000,294.933333,71.866667
4,FOODS_1_002_CA_1,3568.768063,4114.32,545.551937,486.640000,910.080000,471.630000
6,FOODS_1_002_CA_3,471.500610,4995.96,4524.459390,4714.720000,4807.940000,2844.000000
8,FOODS_1_003_CA_1,2404.713795,2703.51,298.796205,476.963333,542.101667,456.237500
...,...,...,...,...,...,...,...
859,FOODS_1_218_CA_4,9215.932797,7412.72,1803.212797,902.253333,2113.533333,1131.981667
860,FOODS_1_219_CA_1,8482.579610,8471.68,10.899610,3297.280000,4352.693333,3441.946667
861,FOODS_1_219_CA_2,5341.540919,6319.04,977.499081,2900.053333,3413.386667,2646.746667
862,FOODS_1_219_CA_3,10065.264303,9027.20,1038.064303,2765.373333,4195.593333,2672.483333


In [162]:
df_metrics_5[['sales_3','sales_6','sales_12']].mean(axis=1)

0      1349.226667
1      2102.924444
2      2353.182222
3       477.804444
4      3491.536667
          ...     
859    8795.309444
860    4774.373333
861    3332.311111
862    5816.050000
863    2272.044444
Length: 864, dtype: float64